In [16]:
#=================================================================
#makesphericalvoxelphantom python 0.5
#This code is the python version of VObjgenerator.m
#Author: Hongzhang(Tommy) Chen
#Imperial College London
#=================================================================

In [16]:
#Function to make and show a spherical phantom containing
#4 nested spheres. The "p" argument will be a structure holding 
#tissue parameters

import numpy as np
import h5py
from ai import cs 
from dataclasses import dataclass

@dataclass
class VObj:
    Gyro: float = 2675.0
    Name: str = 'VObj_Custom'
    Type: str = 'muscle'
    TypeNum: int = 1
    XDim: int = 64
    XDimRes: float = 0.002
    YDim: int = 64
    YDimRes: float = 0.002
    ZDim: int = 64
    ZDimRes: float = 0.002    

def VObjgenerator(p):
    
    # create a meshgrid 
    #x = np.arange(0,64,1)
    #y = np.arange(0,64,1)
    #z = np.arange(0,64,1)
    
    nx, ny, nz= (64, 64, 64)
    xx = np.linspace(0, 1, nx)
    yy = np.linspace(0, 1, ny)
    zz = np.linspace(0, 1, nz)

    x,y,z= np.meshgrid(xx,yy,zz)
    #===========================================================
    #create spheres
    #sphere 1 the big
    r1 = 0.47
    x1 = 0.5
    y1 = 0.5
    z1 = 0.5

    r, theta, phi = cs.cart2sp(x-x1, y-y1, z-z1) #convert cart to spherical

    Vsph1 = r<r1 

    #sphere 2
    r2 = 0.15
    x2 = 0.35
    y2 = 0.35
    z2 = 0.35

    r, theta, phi = cs.cart2sp(x-x2, y-y2, z-z2) #convert cart to spherical

    Vsph2 = r<r2 

    #sphere 3
    r3 = 0.15
    x3 = 0.35
    y3 = 0.65
    z3 = 0.65

    r, theta, phi = cs.cart2sp(x-x3, y-y3, z-z3) #convert cart to spherical

    Vsph3 = r<r3 

    #sphere 4
    r4 = 0.15
    x4 = 0.65
    y4 = 0.35
    z4 = 0.65

    r, theta, phi = cs.cart2sp(x-x4, y-y4, z-z4) #convert cart to spherical

    Vsph4 = r<r4 

    #sphere 5
    r5 = 0.13
    x5 = 0.65
    y5 = 0.35
    z5 = 0.28

    r, theta, phi = cs.cart2sp(x-x5, y-y5, z-z5) #convert cart to spherical

    Vsph5 = r<r5     
    #==========================================================================
    #label each sphere
    #labels will be used to assign tissue properties
    # Note that we have to handle the exterior
    # of the small spheres which lie inside 
    # sphere 1 separately; outside sphere 1 must
    # have the label of 0.

    V1 = np.logical_xor(Vsph1,Vsph2)
    V1 = np.logical_xor(V1,Vsph3)
    V1 = np.logical_xor(V1,Vsph4)
    #Now, assign label 1 to be the exterior of 
    #the small spheres that lie in the big spheres.
    Vsph1 = np.logical_xor(V1,Vsph5)

    #assign tissue properties
 
    T1 = (p['muscle']['t1'])*Vsph1+(p['skin']['t1'])*Vsph2+(p['skin']['t1'])*Vsph3+(p['contissue']['t1'])*Vsph4+(p['muscle']['t1'])*Vsph5;
    T2 = (p['muscle']['t2'])*Vsph1+(p['skin']['t2'])*Vsph2+(p['skin']['t2'])*Vsph3+(p['contissue']['t2'])*Vsph4+(p['muscle']['t2'])*Vsph5;
    T2star = (p['muscle']['t2star'])*Vsph1+(p['skin']['t2star'])*Vsph2+(p['skin']['t2star'])*Vsph3+(p['contissue']['t2star'])*Vsph4+(p['muscle']['t2star'])*Vsph5;
    Rho = (p['muscle']['rho'])*Vsph1+(p['skin']['rho'])*Vsph2+(p['skin']['rho'])*Vsph3+(p['contissue']['rho'])*Vsph4+(p['muscle']['rho'])*Vsph5;
    MassDen = (p['muscle']['mdensity'])*Vsph1+(p['skin']['mdensity'])*Vsph2+(p['skin']['mdensity'])*Vsph3+(p['contissue']['mdensity'])*Vsph4+(p['muscle']['mdensity'])*Vsph5;
    
    #save the data to hdf5
    
    with h5py.File('VObj.h5', 'w')as hf:
        #add tissue properties arrays
        hf.create_dataset('T1', data=T1)
        hf.create_dataset('T2', data=T2)
        hf.create_dataset('T2star', data=T2star)
        hf.create_dataset('Rho', data=Rho)    
        hf.create_dataset('MassDen', data=MassDen)
        
        #add VObj parameters
        hf.create_dataset('Gyro', data=VObj.Gyro)
        hf.create_dataset('Name', data=VObj.Name)
        hf.create_dataset('Type', data=VObj.Type)
        hf.create_dataset('TypeNum', data=VObj.TypeNum)    
        hf.create_dataset('XDim', data=VObj.XDim)
        hf.create_dataset('YDim', data=VObj.YDim)
        hf.create_dataset('ZDim', data=VObj.ZDim)
        hf.create_dataset('XDimRes', data=VObj.XDimRes)
        hf.create_dataset('YDimRes', data=VObj.YDimRes)
        hf.create_dataset('ZDimRes', data=VObj.ZDimRes)        

    hf.close();
    

In [17]:
# tissueinfo as a nested dictionary 
# p contains the properties for different types of tissue
# they will be called to assign to the vitural object later
# note: the parameters are taken from Phantom Parameter Table.xlsx from MRiLab

p = { 'muscle': {'t1': 1.1, 't2': 0.035, 't2star': 0.0175, 'rho': 0.7, 'mdensity': 1090}, 
      'skin': {'t1': 0.3, 't2': 0.03, 't2star': 0.015, 'rho': 0.6, 'mdensity': 1908},
      'contissue':{'t1': 1, 't2': 0.042, 't2star': 0.021, 'rho': 0.75, 'mdensity': 1027}} 

#call function 

VObjgenerator(p);


In [18]:
#check if all parameters are in the h5 file
f = h5py.File('VObj.h5', 'r')
print(list(f.keys()))



['Gyro', 'MassDen', 'Name', 'Rho', 'T1', 'T2', 'T2star', 'Type', 'TypeNum', 'XDim', 'XDimRes', 'YDim', 'YDimRes', 'ZDim', 'ZDimRes']


In [19]:
f['Rho'].shape

(64, 64, 64)

In [20]:
f.close();